In [5]:
import base64
import json
res = "bnVsbA=="

output =base64.b64decode(res).decode('utf-8')
if output == "null":
    output = None

print(output)
print(type(output))

None
<class 'NoneType'>


In [5]:
s3_url = "parent/child/testing/file.pdf"  

import os
def get_file_name(s3_url:str) ->str:
    return os.path.basename(s3_url).split('/')[-1]
   
def change_file_ext(file_name:str, extension:str) ->str:
    file_path = file_name.split(".")
    file_path[-1]=extension
    
    return ".".join(file_path)

file_name = get_file_name(s3_url) 
print(file_name)
updated_file = change_file_ext(file_name, "html")
print(updated_file)

file.pdf
file.html


In [7]:
import pandas as pd
from pandas import DataFrame
from lxml import html

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    tree = html.fromstring(html_content)
    tables = tree.xpath('//table')

    if not tables:
        return None

    tables_html = [html.tostring(table, method="html").decode() for table in tables]

    print(tables_html)
    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            df = pd.read_html(table_html)[0]
            dfs.append(df)
        except ValueError:
            continue

    return dfs if dfs else None
 
def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, 'w') as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n") 
 
with open('sample.html', 'r') as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)

if dfs is not None:
    save_tables_to_markdown(dfs, 'tables.md')



In [26]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import io

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    soup = BeautifulSoup(html_content, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print("There is no table")
        return None

    tables_html = [str(table) for table in tables]

    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            str_buffer = io.StringIO(table_html)
            df = pd.read_html(str_buffer)[0]
            dfs.append(df)
        except ValueError:
            print("There is no table")
            continue
    if dfs:
        return dfs
    return None

def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, "w") as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n")

with open("sample.html") as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)
if dfs is not None:
    save_tables_to_markdown(dfs, "tables.md")


yes


In [2]:
print("| A | B | C | F | G | H | I | K | L | M | N | O | P | Q | R | S | T | U | V | W | X  \n---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---  \nA |  | R' D R U:[U,R' D' R] | R' D R U':[R' D' R,U2] | R:[R D R',U2] td | D:[U,R' D' R] | D':[U,R' D R] |  | [U,R' D' R] | [R D R',U'] | R':[U',R' D' R] tu | R:[R D R',U] td | [R D' R',U'] | [U,R' D R] |  | R':[U2,R' D' R] tu | D':[R D R',U'] | D':[U,R' D' R] | [U,R' D R U' R D' R'] | R F':[D,R' U' R] | R' B:[R U R',D'] | [R D' R' U R' D R,U']  \nB | R' D R U:[R' D' R,U] |  | R' D R U':[R' D' R,U'] | R' F R U D:[R' D' R,U] | U D':[R D' R',U2] | U D:[R D R',U2] | R D R' U D':[R D R',U2] | [U2,R' D' R] ru2 | U:[R D R',U2] | R F R' D':[U2,R' D R] |  | U:[R D' R',U2] | [U2,R' D R] lu2 ru2 | R' D':[R' D R,U2] |  | U D':[R D R',U2] | U D:[R D' R',U2] | [U2,R' D R U' R D' R'] | R F' R' U:[R D R',U2] | R' D' R:[D, R U2 R'] | R D' R':[U2,R' D R]  \nC | R' D R U:[R' D' R,U2] | R' D R U':[U',R' D' R] |  |  | D:[U',R' D' R] | D':[U',R' D R] | R' F' R U:[R' D R,U2] | [U',R' D' R] | U':[U',R D R'] |  | U' R:[U2,R D R'] td | U':[U',R D' R'] | [U',R' D R] | R' D':[R' D R,U'] tu | U' R':[R' D' R,U'] tu | D:[U',R' D R] | D':[U',R' D' R] | [U',R' D R U' R D' R'] | U' R F':[R' U' R,D] | R' B:[R U' R',D'] | R D' R':[U',R' D R]  \nF | R:[U2,R D R'] td | R' F R U2 D:[R' D' R,U'] |  |  | R U':[R' D R,U2] td | R':[F2,R D' R' D] | R U R D' R':[U2,R' D R] td | D R U':[R' D' R,U2] td | R U' D':[R' D R,U2] td |  | R2 D R' U:[R D' R',U] td | R D':[U2,R D R'] td | U R U R':[R' D R,U2] | R D:[U2,R D' R'] td | R2 U' R U':[R' D' R,U2] tu | B R U':[R' D' R,U2] td | R U':[R' D' R,U2] | R U' R' D':[U2,R' D R] | R U' D:[R' D' R,U2] | R:[U2,R D' R'] td | D R:[U2,R D' R'] td  \nG | D:[R' D' R,U] | D:[R' D' R,U2] | D:[R' D' R,U'] | R U:[R' D R,U2] td |  | F:[D,R U' R'] td | U R U' D:[R' D' R,U2] | [R U R',D] td | R' F' R:[U', R D R'] | D R' U':[D',R U R'] | R U:[R' D R,U] | R' F' R:[U',R D' R'] |  | R' F:[D,R U' R'] | R B' R' D:[U,R' D' R] | R' F' R D':[U',R D R'] | R' F' R D':[R' D' R,U] | R U' R':[U2,R' D R] |  | F R U':[U',R D' R'] td | D R:[U',R D' R'] td  \nH | D':[R' D R,U] | U' D:[R D R',U2] | D':[R' D R,U'] | R':[R D' R' D,F2] | F:[R U' R',D] td |  | U' R' U':[R D R',U2] tu |  | U:[D',R U' R'] | D' R' U':[D,R U R'] | R U R':[R' D' R,U'] | U R U R' U:[U,R D' R'] | U:[D,R U' R'] | D' R' U:[D,R U' R'] | U' R' U':[R D R',U'] tu | D' R' U:[D,R U R'] | D R' U R:[R D' R',U'] |  | U' R2 U:[R U R',D] tu | R B R' U:[U,R' D' R] | R D' R' U:[D,R U' R']  \nI |  | R D R' U' D':[R D R',U2] | R' F' R U':[R' D R,U2] | R U R D' R':[R' D R,U2] td | U R U D:[R' D' R,U2] | U' R' U:[R D R',U2] tu |  | R' F' R U':[R' D' R,U2] | U R U:[R' D' R,U2] | R' D' R U' D:[R' D' R,U2] | R B R' U' D':[R D R',U2] | R' F' R:[U2,R D' R'] | U R U:[R' D R,U2] |  | R B R' U':[R D' R',U2] | U' R' U:[R D' R',U2] | D R' F' R:[U2,R D' R'] | D' R B R':[R' D R,U2] | U' R':[U2,R' D R] tu | U' R' U D':[R D R',U2] | R B R' U':[R D R',U2]  \nK | [R' D' R,U] | [R' D' R,U2] | [R' D' R,U'] | D R U:[R' D' R,U2] td | [D,R U R'] td |  | R' F' R U:[R' D' R,U2] |  | R D R':[R' D' R,U] | R' U':[D',R U R'] | D R U:[R' D' R,U] | [D2,R U R'] | U' R U' R':[R' D R,U] | R' U:[D',R U' R'] tu | R B' R':[U,R' D' R] | D' R' F' R:[U',R D R'] | [D',R U R'] |  | R D:[R' D' R,U] td | R:[U',R D' R'] td | D' R U' R':[U2,R' D R]  \nL | [U',R D R'] | U':[R D R',U2] | U':[R D R',U'] | R U D':[R' D R,U2] td | R' F' R:[R D R',U'] | U:[R U' R',D'] | U R U':[R' D' R,U2] | R D R':[U,R' D' R] |  | U' R U':[D,R' U R] | R U D':[R' D R,U] td | R' U R:[R D' R',U'] | R' U:[R U R',D] | U' R U:[D,R' U' R] | R' U R:[R D R',U'] | U':[D',R' U' R] |  | R U' D' R':[U2,R' D R] | U' R':[U,R' D R] tu | U' R' D':[R D R',U'] tu |   \nM | R':[R' D' R,U'] tu | R F R' D':[R' D R,U2] |  |  | D R' U':[R U R',D'] | D' R' U':[R U R',D] | R' D' R U D:[R' D' R,U2] | R' U':[R U R',D'] | U' R U':[R' U R,D] |  | R' F R U:[U,R' D R] | U' R U':[R' U R,D'] | R' U':[R U R',D] | R' F:[D,R U R'] | R' U':[U',R' D' R] tu | R' U' R D':[R D R',U'] | D R' U' R:[R D' R',U'] | R U' R':[U2,R' D' R] | U' R' U:[D,R U' R'] tu | F R:[R D' R',U'] td | R B R' U:[U,R D R']  \nN | R:[U,R D R'] td |  | U' R:[R D R',U2] td | R2 D R' U:[U,R D' R'] td | R U:[U,R' D R] | R U R':[U',R' D' R] | R B R' U D':[R D R',U2] | D R U:[U,R' D' R] td | R U2 D':[R' D R,U'] td | R' F R U:[R' D R,U] |  | R D':[U,R D R'] td | D R U R':[U',R' D' R] | U R':[R' D' R,U'] |  | D2 R U R':[U',R' D' R] | R U:[U,R' D' R] td | R U R2 U':[R U R',D'] | R U2 D:[R' D' R,U'] td | R:[U,R D' R'] td | D R:[U,R D' R']  \nO | [U',R D' R'] | U':[R D' R',U2] | U':[R D' R',U'] | R D':[R D R',U2] td | R' F' R:[R D' R',U'] | U R U R' U:[R D' R',U] | R' F' R:[R D' R',U2] | [R U R',D2] | R' U R:[U',R D' R'] tu | U' R U':[D',R' U R] | R D':[R D R',U] td |  | U' R' F R U':[R D' R',U'] | U' R U:[D',R' U' R] | U2 R U' R' U:[U,R D' R'] |  | U:[R' U2 R,D'] | R U' R' U D':[R D R',U2] | R D R' F:[R U' R',D'] td |  | R D' R':[U,R' D R]  \nP | [R' D R,U] | [R' D R,U2] ru2 | [R' D R,U'] | U R U R':[U2,R' D R] |  | U:[R U' R',D] | U R U':[R' D R,U2] | U' R U' R':[U,R' D R] | R' U:[D,R U R'] | R' U':[D,R U R'] | D R U R':[R' D' R,U'] | U' R' F R U':[U',R D' R'] ru2 |  | R' U:[D,R U' R'] | R B' R':[U,R' D R] | U' R B R':[R' D R,U] | R' U':[D,R U2 R'] tu | U' R' D R:[U',R D' R'] |  | R B R' U2 D:[R' D' R,U'] | F':[D',R U' R'] td  \nQ |  | R' D':[U2,R' D R] | R' D':[U',R' D R] tu | R D:[R D' R',U2] td | R' F:[R U' R',D] | D' R' U:[R U' R',D] |  | R' U:[R U' R',D'] tu | U' R U:[R' U' R,D] | R' F:[R U R',D] | U R':[U',R' D' R] tu | U' R U:[R' U' R,D'] | R' U:[R U' R',D] tu |  | R U R' D':[R' D R,U] | R' D' R' U:[D,R U R'] tu | R U':[D,R' U R] | U R' U':[D',R U R'] | U D R' U':[D',R U R'] | R U':[D',R' U R] | D R U':[D',R' U R]  \nR | R':[R' D' R,U2] tu |  | U' R':[U',R' D' R] tu | R2 U' R U:[R' D' R,U2] tu | R B' R' D:[R' D' R,U] | U' R' U':[U',R D R'] tu | R B R' U:[R D' R',U2] | R B' R':[R' D' R,U] | R' U R:[U',R D R'] | R' U':[R' D' R,U'] tu |  | U2 R U' R' U:[R D' R',U] | R B' R':[R' D R,U] | R U R' D':[U,R' D R] |  | U R' F R:[U',R D' R'] | U' R' U' R:[U,R D R'] | U2 R U R':[R' D' R,U'] | U' R':[U',R' D R] tu | R B:[R' U R,D'] | U R' F R:[U',R D R']  \nS | D':[U',R D R'] | U' D':[R D R',U2] | U' D':[R D R',U'] | B R U:[R' D' R,U2] td | R' F' R D':[R D R',U'] | D' R' U:[R U R',D] | U' R' U':[R D' R',U2] | D' R' F' R:[R D R',U'] | U':[R' U' R,D'] | R' U' R D':[U',R D R'] | D2 R U R':[R' D' R,U'] |  | U' R B R':[U,R' D R] | R' D' R' U:[R U R',D] tu | U R':[F,R2 D' R' D R'] |  | D' R D R':[U,R' D' R] | U2 R U R' U':[R D' R',U'] | U R' F R:[R D' R',U2] |  | U' R' U R:[U2,R D' R']  \nT | D:[U',R D' R'] | U' D:[R D' R',U2] | D':[R' D' R,U'] | R U:[R' D' R,U2] | R' F' R D':[U,R' D' R] | D R' U R:[U',R D' R'] | D R' F' R:[R D' R',U2] | [R U R',D'] td |  | D R' U' R:[U',R D' R'] | R U:[R' D' R,U] td | U:[D',R' U2 R] | R' U':[R U2 R',D] tu | R U':[R' U R,D] | U' R' U' R:[R D R',U] | D' R D R':[R' D' R,U] |  | R U' R' U:[R D R',U2] | R U:[D,R U R'] | R2 U':[R' U' R,D'] tb |   \nU | [R' D R U' R D' R',U] | [R' D R U' R D' R',U2] | [R' D R U' R D' R',U'] | R U' R' D':[R' D R,U2] | R U' R':[R' D R,U2] |  | D' R B R':[U2,R' D R] |  | R U' D' R':[R' D R,U2] | R U' R':[R' D' R,U2] | R U R2 U':[D',R U R'] | R U' R' U' D':[R D R',U2] | U' R' D R:[R D' R',U'] | U R' U':[R U R',D'] | U2 R U R':[U',R' D' R] | U2 R U R' U':[U',R D' R'] | R U' R' U':[R D R',U2] |  | U2 R U R':[U',R' D R] | R U' R' U':[R D' R',U2] | R U' R' U' D:[R D' R',U2]  \nV | R F':[R' U' R,D] | R F' R' U':[R D R',U2] | U' R F':[D,R' U' R] | R U D:[R' D' R,U2] |  | U' R2 U:[D,R U R'] tu | U' R':[R' D R,U2] tu | R D:[U,R' D' R] td | U' R':[R' D R,U] tu | U' R' U:[R U' R',D] tu | R U D:[R' D' R,U] td | R D R' F:[D',R U' R'] td |  | U D R' U':[R U R',D'] | U' R':[R' D R,U'] tu | U R' F R:[U2,R D' R'] | R U:[R U R',D] | U2 R U R':[R' D R,U'] |  | R B R' U:[U,R' D R] | U R' F R:[U2,R D R']  \nW | R' B:[D',R U R'] | R' D' R :[R U2 R',D] | R' B:[D',R U' R'] | R:[R D' R',U2] td | F R U':[R D' R',U'] td | R B R' U:[R' D' R,U] | U' R' U' D':[R D R',U2] | R:[R D' R',U'] td | U' R' D':[U',R D R'] tu | F R:[U',R D' R'] td | R:[R D' R',U] td |  | R B R' U D:[R' D' R,U] | R U':[R' U R,D'] | R B:[D',R' U R] |  | R2 U':[D',R' U' R] tb | R U' R' U:[R D' R',U2] | R B R' U:[R' D R,U] |  | R B:[D,R' U R]  \nX | [U',R D' R' U R' D R] | R D' R':[R' D R,U2] ru2 | R D' R':[R' D R,U'] | D R:[R D' R',U2] td | D R:[R D' R',U'] td | R D' R' U:[R U' R',D] | R B R' U:[R D R',U2] | D' R U' R':[R' D R,U2] |  | R B R' U:[R D R',U] | D R:[R D' R',U] td | R D' R':[R' D R,U] | F':[R U' R',D'] td | D R U':[R' U R,D'] | U R' F R:[R D R',U'] | R':[U R2 D' R2 U',R2] tu |  | R U' R' U D:[R D' R',U2] | U R' F R:[R D R',U2] | R B:[R' U R,D] |   \n|  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n|  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n|  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n| U-Top/D-Side |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n| U-Top/D-Bottom |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n| D-Side/D-Side |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n| LUF/BUR/D-Any |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n| D-Side/D-Bottom |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n| U-Side/D-Any |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n|  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   \n|  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  | \n\n")

| A | B | C | F | G | H | I | K | L | M | N | O | P | Q | R | S | T | U | V | W | X  
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---  
A |  | R' D R U:[U,R' D' R] | R' D R U':[R' D' R,U2] | R:[R D R',U2] td | D:[U,R' D' R] | D':[U,R' D R] |  | [U,R' D' R] | [R D R',U'] | R':[U',R' D' R] tu | R:[R D R',U] td | [R D' R',U'] | [U,R' D R] |  | R':[U2,R' D' R] tu | D':[R D R',U'] | D':[U,R' D' R] | [U,R' D R U' R D' R'] | R F':[D,R' U' R] | R' B:[R U R',D'] | [R D' R' U R' D R,U']  
B | R' D R U:[R' D' R,U] |  | R' D R U':[R' D' R,U'] | R' F R U D:[R' D' R,U] | U D':[R D' R',U2] | U D:[R D R',U2] | R D R' U D':[R D R',U2] | [U2,R' D' R] ru2 | U:[R D R',U2] | R F R' D':[U2,R' D R] |  | U:[R D' R',U2] | [U2,R' D R] lu2 ru2 | R' D':[R' D R,U2] |  | U D':[R D R',U2] | U D:[R D' R',U2] | [U2,R' D R U' R D' R'] | R F' R' U:[R D R',U2] | R' D' R:[D, R U2 R'] | R D' R':[U2,R' D R]  
C | R' D R U:[R' D' R,U2] | R' D R U':[U',R' D' R] |  |  | D:[U',R' D' R] | 